In [1]:
import tweepy, sys, os, logging, json

import pandas as pd
import numpy as np

import pickle

In [2]:
pd.set_option('display.max_columns', 500)

These are the things I need to accomplish (add as needed):
 - Select the columns I need
 - Do any data cleaning necessary for graph analysis
 - Also filter for verified users (add this as a column) to see how many I have.
     - Can also just add the column, and then when I merger everything together, see what the counts are
     - Can also see what counts are if I only focus on users with certain number of followers, users that only retweet less than 90% of the time etc.
     

In [3]:
with open('tweets0.json') as json_file:  
    tweets0_json = json.load(json_file)

In [4]:
tweets0_json[50]

{'created_at': 'Tue Mar 12 04:45:25 +0000 2019',
 'id': 1105328919475965954,
 'id_str': '1105328919475965954',
 'full_text': "RT @carlreiner: I agree wholeheartedly with my fellow Bronxite @AOC's assessment of Ronald Reagan's Anti-American policies. https://t.co/WN…",
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'carlreiner',
    'name': 'carl reiner',
    'id': 637338906,
    'id_str': '637338906',
    'indices': [3, 14]},
   {'screen_name': 'AOC',
    'name': 'Alexandria Ocasio-Cortez',
    'id': 138203134,
    'id_str': '138203134',
    'indices': [63, 67]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {

In [5]:
len(tweets0_json)

4100

In [6]:
tweets0_json[11]

{'created_at': 'Tue Mar 12 04:45:38 +0000 2019',
 'id': 1105328975361073152,
 'id_str': '1105328975361073152',
 'full_text': 'RT @mpukita: .\n.\nI would love to know what countries (outside of the US) AOC has traveled to, where she went in those countries, and what s…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'mpukita',
    'name': 'Mark Pukita',
    'id': 15659178,
    'id_str': '15659178',
    'indices': [3, 11]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 2541890320,
  'id_str': '2541890320',
  'name': 'Lilia Morraz',
  'screen_name': 'LiliaMorraz',
  'location': 'Florida, U

Below I am writing my function to select what data I want to keep for my final analysis


In [7]:
def deefify(tweet_raw):
    outlist = []
    
    for tweet in tweet_raw:
        temp_list = [tweet['id'],
                     tweet['user']['id'],
                     tweet['user']['name'],
                     tweet['user']['screen_name'],
                     tweet['created_at'],
                     tweet['full_text'],
                     tweet['favorite_count'],
                     tweet['retweet_count'],
                     tweet['user']['followers_count'],
                     tweet['user']['verified'],
                     tweet['in_reply_to_status_id'],
                     tweet['in_reply_to_user_id'],
                     tweet['in_reply_to_screen_name']]
        try:
            temp_list.append(tweet['retweeted_status']['full_text'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['retweeted_status']['id'])
        except:
            temp_list.append(np.nan)

        try:
            temp_list.append(tweet['retweeted_status']['user']['id'])
        except:
            temp_list.append(np.nan)
        
        try:
            temp_list.append(tweet['retweeted_status']['user']['screen_name'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['retweeted_status']['user']['verified'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['retweeted_status']['favorite_count'])
        except:
            temp_list.append(np.nan)

        try:
            temp_list.append(tweet['retweeted_status']['user']['followers_count'])
        except:
            temp_list.append(np.nan)         
            
        try:
            temp_list.append(tweet['quoted_status']['full_text'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['quoted_status']['id'])
        except:
            temp_list.append(np.nan)

        try:
            temp_list.append(tweet['quoted_status']['user']['id'])
        except:
            temp_list.append(np.nan)
        
        try:
            temp_list.append(tweet['quoted_status']['user']['screen_name'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['quoted_status']['user']['verified'])
        except:
            temp_list.append(np.nan)
            
        try:
            temp_list.append(tweet['quoted_status']['favorite_count'])
        except:
            temp_list.append(np.nan)
        outlist.append(temp_list)
    
    '''
    tweet_id is the tweet's specific ID (each tweet is unique)
    user_id is each user's specific ID (each user's ID is unique). This is what we can use to link users
    display_name is a user's display name, which can be changed easily and doens't need to be unique. screen_name is unique
    main_text is the text of a tweet. If it's a retweet, it will be stored here, as well as retweet_text, but here, it is truncated
    retweet_text is original tweet, non-truncated
    retweet_id is retweet's tweet_id
    retweet_user_id is the user who was re-tweeted's ID
    retweet_screen_name, obvious
    Same as above for quoted tweets
    not doing coordinates, not enough data there
    '''
    column_names = ['tweet_id',
                    'user_id',
                    'display_name',
                    'screen_name',
                    'tweet_date',
                    'main_text',
                    'fav_count',
                    'rt_count',
                    'followers_count',
                    'is_verified',
                    'reply_to_tweet_id',
                    'reply_to_user_id',
                    'reply_to_screen_name',
                    'retweet_text',
                    'retweet_id',
                    'retweet_user_id',
                    'retweet_screen_name',
                    'retweet_is_verified',
                    'retweet_fav_count',
                    'retweet_followers_count',
                    'quoted_text',
                    'quoted_id',
                    'quoted_user_id',
                    'quoted_screen_name',
                    'quoted_is_verified',
                    'quoted_fav_count']
    
    df_temp = pd.DataFrame(outlist, columns = column_names)
    
    df_temp['is_retweet'] = np.where((df_temp['retweet_text'].isnull()), 0, 1)
    df_temp['is_quote'] = np.where((df_temp['quoted_text'].isnull()), 0, 1)
    return df_temp

In [8]:
df = deefify(tweets0_json)

In [9]:
df.head(6)

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote
0,1105328989005000704,955685794433318912,mythos55,Sandra71005599,Tue Mar 12 04:45:41 +0000 2019,"RT @ARmastrangelo: .@AOC, my entire family imm...",0,5155,232,False,NaN,NaN,None,".@AOC, my entire family immigrated to the Unit...",1.105223e+18,2.652095e+07,ARmastrangelo,True,16041.0,180619.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,1105328986823962625,1523921276,Webcams,webcamsla,Tue Mar 12 04:45:41 +0000 2019,RT @gatewaypundit: Figures: Alexandria Ocasio-...,0,82,1034,False,NaN,NaN,None,Figures: Alexandria Ocasio-Cortez is All Smile...,1.105316e+18,1.921155e+07,gatewaypundit,True,118.0,137906.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,1105328986219929600,730127113261449217,love trump,DGlavine,Tue Mar 12 04:45:40 +0000 2019,"RT @Spring_Harbinge: 🚨hey, @AOC , Is it true y...",0,178,1064,False,NaN,NaN,None,"🚨hey, @AOC , Is it true your failed children's...",1.105232e+18,9.695692e+17,Spring_Harbinge,False,175.0,34404.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,1105328986123468800,16654720,eSublime,eSublime,Tue Mar 12 04:45:40 +0000 2019,@tomselliott @thebradfordfile @AOC @sxsw I liked,0,0,19,False,1.105219e+18,49698174.0,tomselliott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,1105328984030621697,891939067,Kate B,kateb722,Tue Mar 12 04:45:40 +0000 2019,RT @Education4Libs: SHARE THIS EVERYWHERE befo...,0,704,7334,False,NaN,NaN,None,SHARE THIS EVERYWHERE before YouTube takes it ...,1.105315e+18,8.176611e+17,Education4Libs,False,833.0,349574.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
5,1105328982562664448,274207288,Drayton,ScoutDrayton,Tue Mar 12 04:45:40 +0000 2019,RT @NoahPollak: AOC and Ilhan Omar's good frie...,0,1088,11072,False,NaN,NaN,None,AOC and Ilhan Omar's good friend Rep. Rashida ...,1.105199e+18,1.067114e+08,NoahPollak,False,1619.0,23822.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 28 columns):
tweet_id                   4100 non-null int64
user_id                    4100 non-null int64
display_name               4100 non-null object
screen_name                4100 non-null object
tweet_date                 4100 non-null object
main_text                  4100 non-null object
fav_count                  4100 non-null int64
rt_count                   4100 non-null int64
followers_count            4100 non-null int64
is_verified                4100 non-null bool
reply_to_tweet_id          1137 non-null float64
reply_to_user_id           1190 non-null float64
reply_to_screen_name       1190 non-null object
retweet_text               2650 non-null object
retweet_id                 2650 non-null float64
retweet_user_id            2650 non-null float64
retweet_screen_name        2650 non-null object
retweet_is_verified        2650 non-null object
retweet_fav_count          2650 

So of the 4100 tweets in the first tweet file, 2650 are re-tweets

In [11]:
from tweet_clean import deefify2

In [12]:
df_test = deefify2(tweets0_json)

In [16]:
df == df_test

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote
0,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
1,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
2,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
4,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
5,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
7,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
8,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
